<a href="https://colab.research.google.com/github/ferdinand-leitermann/contact-web-scrapping/blob/main/Mp_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import json
import re


from bs4 import BeautifulSoup
import requests

import pickle

#path for saved dataset
path = '/content/gdrive/My Drive/'
dataset_filename = "MP-Dataset.pickle"

domain = "https://maltaprofile.info"
root = "https://directory.maltaprofile.info"

company_index = 0
employee_index = 0

company_list = []
employee_list = []

src_set = set()

class Company:
  def __init__(self, id, name, industry, email, phone, website, fax, employees, address, src):
    self.id = id
    self.name = name
    self.industry = industry
    self.email = email
    self.phone = phone
    self.website = website
    self.fax = fax
    self.address = address
    self.src = src
    if employees is None:
      employees = []
    self.employees = employees
  def add_employee(self, employee):
    self.employees.append(employee)
  def get_employees_ids(self):
    ids = []
    for employee in self.employees:
      ids.append(employee.id)
    return ids

def generate_company_id():
  global company_index
  new_id = 'MP-CO.' + '{:0>4}'.format(company_index)
  company_index += 1
  return new_id
  
class Employee:
  def __init__(self, id, name, position, email, mobile, phone, company):
    self.id = id
    self.name = name
    self.position = position
    self.email = email
    self.mobile = mobile
    self.phone = phone
    self.company = company
    company.add_employee(self)

def generate_employee_id():
  global employee_index
  new_id = 'MP-EE.' + '{:0>4}'.format(employee_index)
  employee_index += 1
  return new_id

class Address:
  def __init__(self, street1, street2, zip, city, country):
    self.street1 = street1
    self.street2 = street2
    self.zip = zip
    self.city = city
    self.country = country

test_set = [['A-Comp',['A-Emp','B-Emp']],['B-Comp',['C-Emp']]]
#c1 = Company("C1","A-Comp",None,None)
#e1 = Employee("E1","A-Emp",c1)
#e2 = Employee('E2', 'B-Emp',c1)
address_test = Address(' ', ' ' ,' ', '','')
#print(c1.id," ",c1.name," ",c1.get_employees_ids())
#print(e1.name, e1.company.id)

html = requests.get(root).text
soup = BeautifulSoup(html, 'lxml')
uls = soup.find_all('ul',{'class':'directory-list'})
for ul in uls:
  lis = ul.findChildren('li',recursive=False)
  for li in lis:
    industry = li.find('a',{'class':'main-cat'}).text.strip()
    #exclude industries for testing
    #if industry != 'Accounting and Auditing':
      #continue 
    subcat = li.find('ul',{'class':'subcat'})
    links = subcat.find_all('a')
    for link in links:
      category_link = domain + link['href']
      category_html = requests.get(category_link).text
      category_soup = BeautifulSoup(category_html,'lxml')
      #print(category_soup.prettify())
      #feature list
      try:
        featurelist = category_soup.find('div',{'id':'feature_listings'})
        feature_cols = featurelist.find_all('div',{'class':'col-sm-4'}) 
        for col in feature_cols:
          company_link = col.find('a')['href']
          src = domain + company_link
          if src not in src_set:
            #print(src)
            src_set.add(src)
            company_html = requests.get(src).text
            company_soup = BeautifulSoup(company_html,'lxml')
            company_info = company_soup.find('div',{'class':'mainProfileinfo featured'})
            street1, street2, zip, city, country, phone, fax, email, website,name = ('',)*10
            #name
            name = company_soup.find('h3',{'class':'company-profile-title pull-right'}).text
            #address
            try:
              address1 = company_info.find('span',{'id':'address_content_1'})
              street1 = address1.text.splitlines()[2]
              city = address1.text.splitlines()[3]
              address = Address(street1, ' ', ' ', city,' ')
            except:
              print('except')
              address = Address('', '', '', '', 'Malta')
            #tel, fax, email
            try:
              address2 = company_info.find('span',{'id':'address_content_2'})
              address2_infos = address2.text.splitlines()
              for info in address2_infos:
                if 'Tel' in info:
                  phone = info 
                if 'Fax' in info:
                  fax = info
                if 'Email' in info:
                  email = info
            except:
              print('address2')
            #website
            try:
              website = company_info.find('a',{'target':'_blank'})['href']
            except:
              print('no website')
            print(name)
            #print(industry)
            #print(phone, '|',fax, '|', email,'|',website)
            #print(address.street1,address.city)
            c_id = generate_company_id()
            company = Company(c_id,name,industry,email,phone,website,fax,None,address,src)
            company_list.append(company)
            
            #key executives
            try:
              executives = company_soup.find('div',{'id':'key-executives-content'}).find_all('div',{'class':'span6 contact-person'})
              for executive in executives:
                try:
                  employee_name = executive.find('span',{'class':'name'}).text
                  position = executive.find('span',{'class':'position'}).text
                  try:
                    email_subject = executive.find('a',onclick=True)['onclick']
                    employee_email = re.search("\'(.*?)\'",email_subject).groups()[0]
                  except:
                    employee_email = None
                  e_id = generate_employee_id()
                  excecutive_employee = Employee(e_id, employee_name, position, employee_email,' ', ' ', company)
                  employee_list.append(excecutive_employee)
                  #print(excecutive_employee.name,excecutive_employee.email, 'added') 
                except Exception as e:
                  print('invalid executive' , e)
            except:
              print('invalid executives')
            #contact persons
            try:
              conatct_persons = company_soup.find('div',{'id':'address_contacts'}).find_all('div',{'class':'col-md-6 contact-person'})
              for person in conatct_persons:
                #contact person
                try:
                  employee_name = person.find('span',{'class':'name main-title-in-dir-profile'}).text
                  position = person.find('span',{'class':'position'}).text
                  #email
                  try:
                    email_subject = person.find('a',onclick=True)['onclick']
                    employee_email = re.search("\'(.*?)\'",email_subject).groups()[0]
                  except:
                    employee_email = None
                  e_id = generate_employee_id()
                  contact_employee = Employee(e_id,employee_name, position, employee_email,' ',' ',company)
                  employee_list.append(contact_employee)
                except:
                  print('invalid contact')
            except:
              print('invalid contact persons')

                #print(name,position,employee_email)

            #print(company_info.prettify())
            #print(company_soup.prettify())
            #hidden_emails = company_soup.find_all('a',onclick=True)
            #for email in hidden_emails:
            #print(email)
      except Exception as e:
        print(e)
        print('no feature')
        #print(category_soup.prettify())
      #list
      try:
        rest_cols = category_soup.find_all('div',{'class':'free-listing'})
        for col in rest_cols:
          company_link = col.find('a')['href']
          src = domain + company_link
          if src not in src_set:
            src_set.add(src)
            try:
              company_html = requests.get(src).text
              company_soup = BeautifulSoup(company_html,'lxml')
              name = street1 = city = phone = fax = email = website= ''
              name = company_soup.find('h3',{'class':'main-title-in-dir-profile'}).text
              try:
                address1 = company_soup.find('span',{'id':'address_content_1'})
                address1_lines = address1.text.splitlines()
                street1 = address1_lines[2]
                city = address1_lines[3]
                address = Address(street1,None,None,city,None)
              except:
                print('invalid adress')
                address = Address(None,None,None,None,None)
              try:
                address2 = company_soup.find('span',{'id':'address_content_2'})
                address2_lines = address2.text.splitlines()
                for line in address2_lines:
                  if 'Tel' in line:
                    phone = line
                  if 'Fax' in line:
                    fax = line
                  if 'Email' in line:
                    email = line
              except:
                print('invalid contact data')
              try:
                website = company_soup.find('address').find('a',{'target':'_blank'})['href']
                #print(company_soup.prettify())
              except:
                print('invalid website')
              print(name)
              c_id = generate_company_id()
              company = Company(c_id,name,industry,email,phone,website,fax,None,address,src)
              company_list.append(company)
            except:
                print('invalid company')
      except Exception as e:
        print('no list', e)
        
        

print(src_set)
print('count', len(src_set))
dataset = [company_list,employee_list]

#save dataset
with open(path + dataset_filename, 'wb') as f:
  pickle.dump(dataset,f,pickle.HIGHEST_PROTOCOL)

for c in dataset[0]:
  print("Company: ", c.id,c.name,c.industry,c.get_employees_ids())
  print('         ',c.address.street1,c.address.city,c.email,c.phone,c.fax,c.website)
for e in dataset[1]:
  print("Employee: ",e.id, e.name,e.position,e.company.id,e.company.name, e.email)

Mounted at /content/gdrive
Nexia BT
M.Meilak & Associates
KPMG
Grant Thornton (Malta)
Capstone Group
HBM Malta Limited
BDO Malta
EY
Baker Tilly Malta
Griffiths + Associates Ltd
KSi Malta
NOUV MT
Credence Corporate and Advisory Services Ltd
D-Consulta Limited
Fact Group
Fairwinds Management Limited
Kosco Management Services Ltd
MGI Malta
Tri-Mer Services Limited
WDM International Ltd
GM Corporate & Fiduciary Services Ltd
Debono & Associates
Finco Trust Group of Companies
PwC (Malta)
PKF (Malta)
Bezzina&Associates
mymaltaaccountant.com
Portman International Limited - Malta
Deloitte
Mahoney & Co
Mazars Malta
Moore Stephens Malta
SG Malta Limited
Charles Scerri & Associates
Parker Rendall Turner
FG Audit Limited
UHY Pace, Galea Musu & Co
ECOVIS Malta Consultants Limited
RSM Malta
Altima Malta Limited
Taxways Group
3a Malta Ltd
EJZ Comprehensive Business Services Ltd.
FinanZ Audit
invalid adress
invalid contact data
invalid website
NM Group
Busuttil & Micallef
DFK Malta
Crowe Horwath (Malta